In [25]:
import os 
import json
from pprint import pprint
import numpy as np

In [54]:
# TODO : label_paths to your own paths 
label_paths = ['label_training/2.book/01.총류/', 'label_training/2.book/02.철학/', 'label_training/2.book/03.종교/', 'label_training/2.book/04.사회과학/']
label_paths

['label_training/2.book/01.총류/',
 'label_training/2.book/02.철학/',
 'label_training/2.book/03.종교/',
 'label_training/2.book/04.사회과학/']

In [58]:
imgs_info = dict()
for src_label_path in label_paths:
    # list all files that end with .json in src_label_path
    src_list = sorted([os.path.join(src_label_path, f) for f in os.listdir(src_label_path) if f.endswith('.json')])
    # store all image info in imgs_info
    
    for idx, src in enumerate(src_list):
        # load json file
        with open(src, 'r', encoding='utf-8-sig') as f:
            data = json.load(f)
        
        # get image info (height, width, tags, license_tag)
        image_info = dict()
        image_info['img_h'] = data['images'][0]['height']
        image_info['img_w'] = data['images'][0]['width']
        image_info['tags'] = None
        image_info['license_tag'] = dict(usability=True, public=True, commercial=True, type='CC-BY-SA',
                                holder=None)
        
        # get word info (points, transcription, language, illegibility, orientation, word_tags)
        words_info = dict()
        for i, word in enumerate(data['annotations']):
            # (x, y, w, h) -> (x1, y1, x2, y2, x3, y3, x4, y4)
            bbox_x = word['bbox'][0]
            bbox_y = word['bbox'][1]
            bbox_w = word['bbox'][2]
            bbox_h = word['bbox'][3]
            items = [bbox_x, bbox_y, bbox_x + bbox_w, bbox_y, bbox_x + bbox_w, bbox_y + bbox_h, bbox_x, bbox_y + bbox_h]
            points = np.array(items[:], dtype=np.float32).reshape(4, 2).tolist()
            
            # get transcription
            transcription = word['text']
            # get language (default: 'ko')
            language = 'ko'
            # get illegibility (if transcription == 'xxx', illegibility = True)
            illegibility = transcription == 'xxx'
            # get orientation by wordorientation
            orientation = "Horizontal" if data['metadata'][0]['wordorientation'] == "가로" else "Vertical"
            
            words_info[i] = dict(
                points=points, transcription=transcription, language=[language],
                illegibility=illegibility, orientation=orientation, word_tags=None
            )
        image_info['words'] = words_info
        
        img_path = data['images'][0]['file_name']
        imgs_info[img_path] = image_info

In [59]:
# TODO : need to change path to your own path (first path)
with open(os.path.join('label_training/2.book', 'train.json'), 'w', encoding = 'utf-8-sig') as f:
    anno = dict(images = imgs_info)
    json.dump(anno, f, indent=4)